# Information Searching

Vector로 정보 표현: 계산이 빠르고 알아보기 쉬우므로 vector로 정보를 표현한다.

In [1]:
import requests
from bs4 import BeautifulSoup

In [9]:
headers = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "accept-encoding":"false"
}
params = {
    
}

In [3]:
def getUrl_global(url, params=None, select="a", num_retries=2):
    html = requests.get(url, params=params, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getUrl_global(url, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, "lxml")
    links = dom.select(select)
    
    return [link["href"] for link in links if link.has_attr("href") == True]

In [4]:
def getContents(url, params=None, select="a", num_retries=2):
    html = requests.get(url, params=params, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getContents(url, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, "html.parser")
    links = dom.select(select)
    
    return links

In [10]:
seed = 'http://media.daum.net/'
select = "div.box_headline a"

contentList = getUrl_global(seed, params, select)
contentList = contentList[1:]
result = list()

In [11]:
for url in contentList:
    docid = url[-17:]
    content = getContents(url, params, "div.news_view p")
    with open ('/Users/keith_lee/Desktop/doc/{0}'.format(docid), 'w') as fp:
        for i in range(len(content)):
            fp.write(content[i].get_text().strip())
            fp.write(' ')

media.daum.net에서 뉴스들을 가져와 기사 내용만 저장한다.

In [12]:
import glob
import re

In [13]:
fileList = glob.glob('/Users/keith_lee/Desktop/doc/*')

In [15]:
with open(fileList[0], 'r') as fp:
    content = fp.read()
    content = re.sub(r'[\s]{2,}', "", content)

In [16]:
import nltk

In [17]:
for sentence in nltk.sent_tokenize(content):
    words = nltk.word_tokenize(sentence)
    
    for word in words:
        tokens = []
        for i in range(len(word)):
            if i+1 < len(word):
                tokens.append(word[i:i+2])

문장을 단어 단위로 자르고 잘라낸 단어를 2글자 단위로 자른다.

In [18]:
from konlpy.tag import Kkma

In [19]:
for sentence in nltk.sent_tokenize(content):
    words = Kkma().nouns(sentence)
    print(sentence)
    print(words)

 (서울=연합뉴스) 박초롱 기자 = 서울의 1일 낮 기온이 38.5도를 넘으며 기상관측 111년 만에 역대 최고기록을 갈아치운 가운데 오존까지 시민들의 건강을 위협하고 있다.
['서울', '연합', '연합뉴스', '뉴스', '박', '박초롱', '초롱', '기자', '1', '1일', '일', '낮', '기온', '38.5', '38.5도', '도', '기상', '기상관측', '관측', '111', '111년', '년', '만', '역대', '최고', '최고기록', '기록', '가운데', '오존', '시민', '건강', '위협']
서울시는 이날 오후 1시 기준으로 도심(중로·중·용산구)과 동북권(성동, 광진, 동대문, 중랑, 성북, 강북, 도봉, 노원구), 서남권(양천·강서·구로·금천·영등포·동작·관악구)에 오존 주의보를 발령했다.
['서울시', '이날', '오후', '1', '1시', '시', '기준', '도심', '중로', '중', '용산', '용산구', '구', '과', '동북', '성동', '광진', '동대문', '중랑', '성북', '강북', '도봉', '노원', '노원구', '서남권', '양천', '강서', '구로', '금', '금천', '천', '영등포', '동작', '관악', '관악구', '오존', '주의보', '발령']
이어 오후 2시를 기해 동남권(서초·강남·송파·강동)에도 오존 주의보가 발령됐다.
['오후', '2', '2시', '시', '동남', '서', '서초', '초', '강남', '송', '송파', '파', '강동', '오존', '주의보', '발령']
중구에서 측정한 오존 농도는 0.123ppm으로 나타났다.
['중구', '측정', '오존', '농도', '0.123']
동대문구의 오존 농도는 0.131ppm, 강서구 0.120ppm, 서초구는 0.120ppm이었다.
['동대문', '동대문구', '구', '오존', '농도', '0.131', '강서', '강서구', '구', '0.120', '서초', '서초구']
서울시는 1시간 평균 오

Kkma를 활용하여 모든 문장에 대해 형태소 분석을 실시한 후 명사만 뽑아낸다.

=> 문제점: 중복되어 등장하는 명사는 추출되지 않는다 = 단어 등장 횟수를 알 수 없다.

___

## Forward Index

In [21]:
uniqNouns = []

for sentence in nltk.sent_tokenize(content):
    nouns = []
    for word in nltk.word_tokenize(sentence):
        nouns.extend(Kkma().nouns(word))
        nouns = list(set(nouns))
    uniqNouns.extend(nouns)
    uniqNouns = list(set(uniqNouns))
print(len(uniqNouns))

132


파일 하나를 읽어 명사만 뽑아냈다.

In [22]:
uniqNouns = []

for file in fileList:
    with open(file, 'r') as fp:
        content = fp.read()
        content = re.sub(r'[\s]{2,}', '', content)
        
    for sentence in nltk.sent_tokenize(content):
        nouns = []
        for word in nltk.word_tokenize(sentence):
            nouns.extend(Kkma().nouns(word))
            nouns = list(set(nouns))
        uniqNouns.extend(nouns)
        uniqNouns = list(set(uniqNouns))
            
print(len(uniqNouns))

2633


디렉토리 내 모든 파일을 읽어 명사만 뽑아냈다.

In [23]:
docNouns = {}

for file in fileList:
    docid = file[-17:]
    
    with open(file, 'r') as fp:
        content = fp.read()
        content = re.sub(r'[\s]{2,}', '', content)
        
    nouns = {}
    
    for sentence in nltk.sent_tokenize(content):
        for word in nltk.word_tokenize(sentence):
            for w in Kkma().nouns(word):
                if w in nouns.keys():
                    nouns[w] += 1
                else:
                    nouns[w] = 1
    
    docNouns[docid] = nouns

새로운 명사를 찾으면 count를 1로 설정하여 추가하고 이미 있는 명사일 경우 count를 늘린다.

In [24]:
for (k, v) in docNouns.items():
    print('{0} - {1}'.format(k, len(v)))

20180801143528088 - 132
20180801142859853 - 128
20180807212450487 - 203
20180807213506621 - 128
20180801143420047 - 148
20180801142952883 - 183
20180807212413479 - 129
20180807215225853 - 114
20180807220127013 - 219
20180807212110439 - 156
20180807211912409 - 146
20180807220049001 - 127
20180807212553502 - 111
20180801141138247 - 96
20180801150201022 - 110
20180801144439453 - 109
20180801144119329 - 211
20180801141712421 - 120
20180807213757650 - 171
20180801144437449 - 0
20180807212552501 - 149
20180801145118652 - 232
20180801141549383 - 213
20180807213426606 - 117
20180801145958922 - 137
20180807220157018 - 167
20180807212538495 - 125
20180801145951917 - 206


Vector에 값을 채우는 과정을 실행하였다.

In [25]:
docNouns = {}
maxCount = {}

for file in fileList:
    docid = file[-17:]
    
    with open(file, 'r') as fp:
        maxCount[docid] = 0
        content = fp.read()
        content = re.sub(r'[\s]{2,}', '', content)
        
    nouns = {}
    
    for sentence in nltk.sent_tokenize(content):        
        for word in nltk.word_tokenize(sentence):
            for w in Kkma().nouns(word):
                if w in nouns.keys():
                    nouns[w] += 1
                else:
                    nouns[w] = 1
                if maxCount[docid] < nouns[w]:
                    maxCount[docid] = nouns[w]
    
    docNouns[docid] = nouns

각 파일들에서 가장 많이 등장한 명사의 등장 횟수를 maxCount에 저장하였다.

In [26]:
for (k, v) in docNouns.items():
    print('{0} - {1}/{2}'.format(k, len(v), maxCount[k]))

20180801143528088 - 132/9
20180801142859853 - 128/10
20180807212450487 - 203/7
20180807213506621 - 128/7
20180801143420047 - 148/11
20180801142952883 - 183/8
20180807212413479 - 129/6
20180807215225853 - 114/10
20180807220127013 - 219/22
20180807212110439 - 156/9
20180807211912409 - 146/9
20180807220049001 - 127/18
20180807212553502 - 111/9
20180801141138247 - 96/5
20180801150201022 - 110/6
20180801144439453 - 109/5
20180801144119329 - 211/7
20180801141712421 - 120/6
20180807213757650 - 171/10
20180801144437449 - 0/0
20180807212552501 - 149/10
20180801145118652 - 232/19
20180801141549383 - 213/16
20180807213426606 - 117/7
20180801145958922 - 137/10
20180807220157018 - 167/17
20180807212538495 - 125/11
20180801145951917 - 206/9


___

## Inverted Index

In [27]:
invertedIndex = {}

for noun in uniqNouns:
    invertedIndex[noun] = []
    
    for k, v in docNouns.items():
        if noun in v:
            invertedIndex[noun].append(k)

In [28]:
print(invertedIndex)

{'어디': ['20180807220127013'], '777기종': ['20180807213757650'], '운항': ['20180801141549383'], '헌법소원': ['20180807211912409'], '실제': ['20180801145118652'], '천의': ['20180801141138247'], '김정': ['20180807213426606'], '유력': ['20180801150201022', '20180801145118652'], '도내': ['20180807215225853'], '당부': ['20180801145958922'], '전체': ['20180801145118652', '20180807220157018'], '수의계약': ['20180801142859853'], '접경지대': ['20180807212450487'], '고용': ['20180807212538495'], '총재': ['20180807220049001'], '2시': ['20180801143528088', '20180801141138247'], '영부인': ['20180807212110439'], '스카이': ['20180801141549383'], '은행': ['20180807220049001', '20180807212553502'], '우수': ['20180801142859853'], '발견': ['20180807212450487', '20180807220049001', '20180801145958922'], '태풍': ['20180801145951917'], '만큼': ['20180807213506621', '20180801141549383', '20180807212538495'], '박지혜': ['20180807212413479'], '잘못': ['20180801142859853', '20180807211912409'], '경기': ['20180801141712421'], '파악': ['20180801142859853', '201808011459589

각 명사들이 어느 문서에서 등장했는지 알 수 있다.

___

## TF-IDF

In [29]:
kRation = 0.5
TF = {}

for k, v in docNouns.items():
    tfList = {}
    
    for w in v:
        tfList[w] = kRation + (1-kRation) * (v[w] / maxCount[k])
        
    TF[k] = tfList

TF 계산식을 모든 파일의 명사 리스트에 적용하고 결과를 저장하였다.
- v[w]: 등장 빈도
- v[w] / maxCount[k]: TF(Term Frequency)

In [30]:
from math import log10

docSize = len(fileList)
TFIDF = {}

for k, v in TF.items():
    idfList = {}
    
    for t in v:
        idf = log10(docSize / len(invertedIndex[t]))
        idfList[t] = v[t] * idf
        
    TFIDF[k] = idfList

디렉토리 내 모든 파일들의 명사 리스트 전체에 대해 TF-IDF를 계산한 결과를 저장하였다.

In [33]:
query = """
내가 이리 견딜 수 없게 열이 심하게 나는 까닭은 
하고픈 말이 너무 많기 때문입니다.
어제 저녁 처음 만난 그와 급히 인사를 나눈 까닭은 
흙투성이 손을 들키고 싶지 않았기 때문입니다.

내 이름의 마지막 글자로 나를 불러주길 원한 까닭은
이 작은 별이 내겐 너무 외롭기 때문입니다.
내가 이리 높은 목소리로 노래를 부르는 까닭은
당신에게 전해야 할 것들이 너무 많이 쌓였기 때문입니다.

아아
이런 두서없는 말들을
아아 
하얀 새하얀 종이에 써서
아아
낯선 곳에 있는 우체통에 넣고
아아
누가 볼 새라 나는 도망쳐버렸네.


언젠가 너의 목마름이 그치면
언젠가 나의 목마름이 그치면

아아
제대로 도착했으려나
아아
글씨 못알아보면 어쩌나
아아
읽지도 않고 버리면 어떡하나
아아
이런 걱정에 나는 밤을 새 버렸네.
"""

queryNouns = {}
maxCount = 0
    
for word in nltk.word_tokenize(query):
    for w in Kkma().nouns(word):
        if w in queryNouns.keys():
            queryNouns[w] += 1
        else:
            queryNouns[w] = 1
        if maxCount < queryNouns[w]:
            maxCount = queryNouns[w]
            
print(len(queryNouns), maxCount)
print(queryNouns)

39 4
{'수': 1, '나': 3, '까닭': 4, '말': 2, '때문': 4, '어제': 1, '저녁': 1, '처음': 1, '그': 1, '인사': 1, '흙': 1, '손': 1, '내': 2, '이름': 1, '마지막': 1, '글자': 1, '원한': 1, '이': 1, '별': 1, '내겐': 1, '겐': 1, '목소리': 1, '노래': 1, '당신': 1, '종이': 1, '곳': 1, '우체통': 1, '볼': 1, '새라': 1, '너': 1, '목': 2, '목마름': 2, '마름': 2, '나의': 1, '도착': 1, '글씨': 1, '걱정': 1, '밤': 1, '새': 1}


검색하고자 하는 문장에서 명사만 추려냈다.

In [35]:
queryWeight = {}

for k, v in queryNouns.items():
    queryWeight[k] = kRation + (1-kRation) * (v / maxCount)
    
queryWeight

{'수': 0.625,
 '나': 0.875,
 '까닭': 1.0,
 '말': 0.75,
 '때문': 1.0,
 '어제': 0.625,
 '저녁': 0.625,
 '처음': 0.625,
 '그': 0.625,
 '인사': 0.625,
 '흙': 0.625,
 '손': 0.625,
 '내': 0.75,
 '이름': 0.625,
 '마지막': 0.625,
 '글자': 0.625,
 '원한': 0.625,
 '이': 0.625,
 '별': 0.625,
 '내겐': 0.625,
 '겐': 0.625,
 '목소리': 0.625,
 '노래': 0.625,
 '당신': 0.625,
 '종이': 0.625,
 '곳': 0.625,
 '우체통': 0.625,
 '볼': 0.625,
 '새라': 0.625,
 '너': 0.625,
 '목': 0.75,
 '목마름': 0.75,
 '마름': 0.75,
 '나의': 0.625,
 '도착': 0.625,
 '글씨': 0.625,
 '걱정': 0.625,
 '밤': 0.625,
 '새': 0.625}

검색하고자 하는 문장에 대해 TF-IDF 분석을 실행하였다.

In [36]:
from math import sqrt

docLength = {}

for k, v in TFIDF.items():
    sumPow = 0.0
    
    for t in v:
        sumPow += v[t]**2
        
    docLength[k] = sqrt(sumPow)

In [37]:
docLength

{'20180801143528088': 8.0467771804708,
 '20180801142859853': 7.360798424859128,
 '20180807212450487': 10.996457029865976,
 '20180807213506621': 8.07410884404419,
 '20180801143420047': 8.401704369302063,
 '20180801142952883': 10.196387626287043,
 '20180807212413479': 8.09851944322959,
 '20180807215225853': 7.391134081912768,
 '20180807220127013': 9.875502662744024,
 '20180807212110439': 8.785661111424611,
 '20180807211912409': 8.624802034132479,
 '20180807220049001': 7.1493142922010104,
 '20180807212553502': 7.58467613244686,
 '20180801141138247': 6.9085394807300755,
 '20180801150201022': 7.073258627436703,
 '20180801144439453': 7.865658915011207,
 '20180801144119329': 10.697281295266015,
 '20180801141712421': 7.932083011045924,
 '20180807213757650': 9.193279723618874,
 '20180801144437449': 0.0,
 '20180807212552501': 8.317911284395034,
 '20180801145118652': 10.249017834675476,
 '20180801141549383': 9.836318595302366,
 '20180807213426606': 7.155294958159432,
 '20180801145958922': 7.93044

문서의 벡터 크기가 클수록 많은 단어가 들어있거나 가중치가 높은 단어가 들어있다.

In [38]:
for k, v in queryWeight.items():
    if k in invertedIndex.keys():
        print(k)
        invertedIndex[k]

수
나
말
때문
어제
처음
그
인사
내
이름
이
별
목소리
당신
곳
볼
도착
밤
새


입력된 문구에서 추출한 명사 중 파일에서 가져온 명사 리스트에 있는 명사들만 남겼다.

In [39]:
result = {}

for k, v in queryWeight.items():
    if k in invertedIndex.keys():
        
        for docid in invertedIndex[k]:
            if docid in result.keys():
                result[docid] += TFIDF[docid][k] * queryWeight[k]
            else:
                result[docid] = TFIDF[docid][k] * queryWeight[k]
            
for k, v in result.items():
    result[k] = v / docLength[k]
    
result = sorted(result.items(), key=lambda k:k[1], reverse=True)
    
for k, v in result:
    print('[{0}] 유사도: {1}'.format(k, v))
    with open('/Users/keith_lee/Desktop/doc/'+k, 'r') as fp:
        print(fp.readlines(10))

[20180801144439453] 유사도: 0.2805676907527187
[' (서울=뉴스1) 이원준 기자 = 니콜라스 마두로 베네수엘라 대통령이 물가상승과 식량부족에 신음하고 있는 자국 경제 상황에 대해 "실패했다"고 인정했다. AFP통신에 따르면 마두로 대통령은 31일(현지시간) 카라카스에서 열린 집권당 통합사회주의당(PSUV) 의원총회에 참석해 "지금까지 우리가 시도해 온 생산모델은 실패했다"며 이같이 밝혔다. 그는 이어 "그 책임은 우리들, 나와 당신의 몫"이라고 덧붙였다. 마두로 대통령은 다만 실패를 인정하면서도 새로운 도약을 다짐했다. 그는 "외부 침략이나 (미국의) 봉쇄 여부와 관계없이, 우리는 베네수엘라를 경제 강국으로 만들어야 한다"며 "더 이상의 징징거림은 없다. 나는 해결책을 원한다"고 강조했다. 베네수엘라는 지난 2013년 마두로 정권 출범 후 생필품 가격 폭등, 실업률 상승 등 경제난에 신음하고 있다. 이에 마두로 대통령은 가격 통제 정책과 자국 통화를 10만대 1로 액면절하 하는 화폐개혁안까지 내놓았지만 물가는 종잡을 수 없이 뛰고 있는 상황이다. 국제통화기금(IMF)은 올해 베네수엘라의 물가 상승률이 무려 100만%에 달할 것이라고 예측하고 있다. 경제 성장률은 마이너스(-)18%일 것으로 전망된다. wonjun44@news1.kr ']
[20180807220127013] 유사도: 0.20488023475905154
['[앵커] 도심의 불법주차가 폭염으로 더 기승을 부리고 있습니다. "더워서 잠깐 세웠다"며 날씨 탓을 하지만 인근의 주민들은 괴로워하고 있습니다. 이 곳은 마포구 연남동입니다. 최근 골목길 상권이 발달하면서 많은 사람들이 찾는 곳인데요. 하지만 상권이 커질수록 더 골칫거리가 되는 문제도 있습니다. 바로 이렇게 골목길 곳곳에 주차된 불법주차 차량들입니다. 작은 음식점이 자리잡은 골목골목마다 차들이 세워져 있습니다. 양방향 도로지만 차 1대가 지나가기도 빠듯합니다. 골목 진입을 아예 포기하는 차도 있습니다. 불법 주차

TF-IDF 통계를 기반으로 벡터로 표현된 문서 모델을 활용하여 간단한 검색엔진을 만들었다.